In [1]:
from wikitables import *
import mwclient
import mwparserfromhell as mwp
import logging

In [2]:
logger = logging.getLogger("wikitables")
logger.setLevel(logging.DEBUG)

In [3]:
# retrieve list of articles with tables in wikipedia
# get 10 articles each from a-z with appeneded random letter to act as random sample of tables
import string
import random

articles = list()
site = mwclient.Site('en.wikipedia.org')

# for letter in string.ascii_lowercase:
#     counter = 0
#     while (counter!=10):
#         for page in site.allpages(filterredir='nonredirects', prefix=letter+random.choice(string.ascii_lowercase)):
#             tables = mwp.parse(page.text()).filter_tags(matches=ftag('table'))
#             if tables:
#                 t = import_tables(page.name)
#                 if t:
#                     print(page.name)
#                     articles.append(page.name)
#                     counter+=1
#                     break

In [4]:
# # save the articles with tables
# import pickle

# with open("articles.txt", "wb") as fp:
#     pickle.dump(articles, fp)

In [5]:
# import the saved articles
import pickle
with open("articles.txt", "rb") as fp:
    articles = sorted(list(set(pickle.load(fp))))

In [33]:
# extract tables and features
t = import_tables(articles[201])
print(t)

DEBUG:wikitables:Zydis|Table 0: found header at row 0 (4 <th> elements)
DEBUG:wikitables:Zydis|Table 0: parsed 10 rows 4 cols


defaultdict(<function <lambda> at 0x7f71bc0a8d90>, {'extract': 'Zydis is a technology used to manufacture orally disintegrating tablets developed by R.P. Scherer Corporation. Zydis tablets dissolve in the mouth within 3 seconds.', 'title': 'Zydis', 0: defaultdict(<function <lambda> at 0x7f71bc0a8d90>, {'table': defaultdict(<function <lambda> at 0x7f71bc0a8d90>, {0: <WikiTable 'Zydis|Table 0'>}), 'head': ' Fast dissolving drugs with Zydis technology ', 'text': 'Data from "Fast Disintegrating Drug Delivery Systems: A Review with Special Emphasis on Fast Disintegrating Tablets" (2013).'})})


In [7]:
# disambiguate entities within cells
import tagme

with open("tagme", 'r') as file:
    token = file.readline().strip()
    
tagme.GCUBE_TOKEN = token

lunch_annotations = tagme.annotate("My favourite meal is Mexican burritos.")

# Print annotations with a score higher than 0.1
for ann in lunch_annotations.get_annotations(0.1):
    print(ann)


meal -> Meal (score: 0.2014230340719223)
Mexican -> Mexican cuisine (score: 0.36614900827407837)
burritos -> Burrito (score: 0.28607892990112305)


In [32]:
# extract RDF triples
# use SPARQL queries
for k,v in t.items():
    if type(v) is defaultdict:
        for k1,v1 in v.items():
            if type(v1) is defaultdict:
                for k2,v2 in v1.items():
                    print(v2.rows)

# only extract triples that don't timeout

from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""SELECT * WHERE
{
     wd:Q2 ?p ?o .
     FILTER(STRSTARTS(str(?p), "http://www.wikidata.org/prop/direct/"))
     SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result)


[{'Trade name': Ativan, 'Active formula': Lorazepam, 'Manufacturer': Valeant Pharmaceuticals, 'Indication': Anxiety disorders}, {'Trade name': Claritin RediTab, 'Active formula': Loratadine, 'Manufacturer': Schering-Plough, 'Indication': Allergy}, {'Trade name': Imodium, 'Active formula': Loperamide, 'Manufacturer': Johnson & Johnson, 'Indication': Diarrhea}, {'Trade name': Feldene melt, 'Active formula': Piroxicam, 'Manufacturer': Pfizer, 'Indication': Pain relief}, {'Trade name': Maxalt MLT, 'Active formula': Rizatriptan, 'Manufacturer': Merck & Co., 'Indication': Headache}, {'Trade name': Motilium, 'Active formula': Domperidone, 'Manufacturer': Johnson & Johnson, 'Indication': Nausea and vomiting}, {'Trade name': Zelapar, 'Active formula': Selegiline, 'Manufacturer': Valeant Pharmaceuticals, 'Indication': Parkinson's disease , Major depressive disorder}, {'Trade name': Pepcid RPD, 'Active formula': Famotidine, 'Manufacturer': Johnson & Johnson, 'Indication': Peptic ulcer disease (PU